In [ ]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

# --- 1. Preparazione Dati ---
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# --- 2. Definizione della "Objective Function" ---
# Questa è la funzione che Optuna chiamerà centinaia di volte.
# Deve restituire UN solo numero: il valore da ottimizzare (es. Accuracy).

def objective(trial):
    # A. Suggerimento Parametri (Search Space)
    # Optuna sceglie i valori in modo intelligente basandosi sulla storia passata
    params = {
        'objective': 'binary:logistic',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # Intero tra 100 e 1000
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True), # Scala logaritmica
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0), # % feature per albero
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1 Reg (Lasso)
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2 Reg (Ridge)
        'tree_method': 'hist',
        'device': 'gpu',
        'n_jobs': -1,
        'random_state': 42,
        # Early Stopping nel costruttore per versioni recenti (o gestito in fit)
    }
    
    
    # B. Creazione Modello
    model = xgb.XGBClassifier(**params)
    
    # C. Validazione Robusta (Cross-Validation)
    # Usiamo 5-Fold per essere sicuri che il risultato non sia fortuna
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    
    # Restituiamo la media dell'accuratezza
    return scores.mean()

# --- 3. Creazione dello Studio ---
print("--- Inizio Ottimizzazione Bayesiana ---")
# direction='maximize' perché vogliamo ALTA accuratezza (se fosse RMSE useremmo minimize)
study = optuna.create_study(direction='maximize', study_name="XGBoost_BreastCancer")

# Lanciamo l'ottimizzazione (es. 50 tentativi)
# Optuna mostrerà una progress bar con i risultati in tempo reale
study.optimize(objective, n_trials=50, show_progress_bar=True)

# --- 4. Risultati ---
print("\n--- Risultati Ottimizzazione ---")
print(f"Miglior Trial (Tentativo #{study.best_trial.number})")
print(f"Accuratezza Migliore (CV): {study.best_value:.4f}")
print("Migliori Iperparametri:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# --- 5. Validazione Finale sul Test Set ---
# Ricostruiamo il modello con i parametri vincenti
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, n_jobs=-1, random_state=42)
final_model.fit(X_train, y_train)
final_acc = final_model.score(X_test, y_test)

print(f"\nAccuratezza Finale sul Test Set: {final_acc:.4f}")

[I 2026-01-06 19:51:12,218] A new study created in memory with name: XGBoost_BreastCancer


--- Inizio Ottimizzazione Bayesiana ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2026-01-06 19:51:12,974] Trial 0 finished with value: 0.9670329670329669 and parameters: {'n_estimators': 431, 'learning_rate': 0.046126194303366744, 'max_depth': 9, 'subsample': 0.9826452144281044, 'colsample_bytree': 0.5997488722531131, 'reg_alpha': 5.588442879808919e-05, 'reg_lambda': 5.423099576094846}. Best is trial 0 with value: 0.9670329670329669.
[I 2026-01-06 19:51:14,660] Trial 1 finished with value: 0.9604395604395604 and parameters: {'n_estimators': 549, 'learning_rate': 0.005540913426134001, 'max_depth': 10, 'subsample': 0.5903986727471358, 'colsample_bytree': 0.9211118105074912, 'reg_alpha': 0.00011764987664583178, 'reg_lambda': 8.430663132589746e-07}. Best is trial 0 with value: 0.9670329670329669.
[I 2026-01-06 19:51:15,920] Trial 2 finished with value: 0.956043956043956 and parameters: {'n_estimators': 574, 'learning_rate': 0.0028135694225538636, 'max_depth': 5, 'subsample': 0.5433970779139308, 'colsample_bytree': 0.6711770492473093, 'reg_alpha': 0.00044924748984913

In [ ]:
import optuna
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from optuna.integration import XGBoostPruningCallback

# =========================
# 1. DATA PREPARATION
# =========================
data = load_breast_cancer()
X, y = data.data, data.target

# Train / Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Train / Validation (FIXED split for Optuna)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

# =========================
# 2. OPTUNA OBJECTIVE
# =========================
def objective(trial):

    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "n_estimators": 4000,  # upper bound
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "tree_method": "hist",
        "device": "gpu",
        "random_state": 42,
        "n_jobs": -1,
        "early_stopping_rounds": 100
    }

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)


    pruning_callback = XGBoostPruningCallback(trial, "validation_0-logloss")

    bst = xgb.train(
        params,
        dtrain,
        num_boost_round = 4000,
        evals=[(dvalid, "validation_0" )],
        early_stopping_rounds = 100,
        callbacks=[pruning_callback],
        verbose_eval = False
    )

    # Validation metric for Optuna
    y_pred_proba = bst.predict(dvalid)
    return log_loss(y_val, y_pred_proba)

# =========================
# 3. OPTUNA STUDY
# =========================
study = optuna.create_study(
    direction="minimize",
    study_name="XGBoost_BreastCancer"
)

study.optimize(objective, n_trials=50, show_progress_bar=True)

# =========================
# 4. RESULTS
# =========================
print("\nBest trial:", study.best_trial.number)
print("Best validation logloss:", study.best_value)
print("Best hyperparameters:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

# =========================
# 5. FINAL MODEL (REFIT)
# =========================
final_model = xgb.XGBClassifier(
    **study.best_params,
    n_estimators=4000,
    early_stopping_rounds=100,
    tree_method="hist",
    device="gpu",
    random_state=42,
    n_jobs=-1
)

final_model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

y_test_pred = final_model.predict(X_test)
final_acc = accuracy_score(y_test, y_test_pred)

print(f"\nFinal Test Accuracy: {final_acc:.4f}")


[I 2026-01-06 22:18:20,178] A new study created in memory with name: XGBoost_BreastCancer


  0%|          | 0/50 [00:00<?, ?it/s]

[0]	validation_0-logloss:0.65000
[1]	validation_0-logloss:0.63936
[2]	validation_0-logloss:0.62883
[3]	validation_0-logloss:0.61960
[4]	validation_0-logloss:0.60982
[5]	validation_0-logloss:0.60020
[6]	validation_0-logloss:0.59120
[7]	validation_0-logloss:0.58214
[8]	validation_0-logloss:0.57321
[9]	validation_0-logloss:0.56457
[10]	validation_0-logloss:0.55648
[11]	validation_0-logloss:0.54811
[12]	validation_0-logloss:0.54052
[13]	validation_0-logloss:0.53243
[14]	validation_0-logloss:0.52512
[15]	validation_0-logloss:0.51762
[16]	validation_0-logloss:0.51072
[17]	validation_0-logloss:0.50346
[18]	validation_0-logloss:0.49663
[19]	validation_0-logloss:0.49053
[20]	validation_0-logloss:0.48384
[21]	validation_0-logloss:0.47760
[22]	validation_0-logloss:0.47149
[23]	validation_0-logloss:0.46516
[24]	validation_0-logloss:0.45965
[25]	validation_0-logloss:0.45367
[26]	validation_0-logloss:0.44852
[27]	validation_0-logloss:0.44290
[28]	validation_0-logloss:0.43754
[29]	validation_0-loglos

c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "early_stopping_rounds", "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[108]	validation_0-logloss:0.21379
[109]	validation_0-logloss:0.21220
[110]	validation_0-logloss:0.21068
[111]	validation_0-logloss:0.20928
[112]	validation_0-logloss:0.20792
[113]	validation_0-logloss:0.20668
[114]	validation_0-logloss:0.20575
[115]	validation_0-logloss:0.20481
[116]	validation_0-logloss:0.20357
[117]	validation_0-logloss:0.20233
[118]	validation_0-logloss:0.20111
[119]	validation_0-logloss:0.20033
[120]	validation_0-logloss:0.19949
[121]	validation_0-logloss:0.19808
[122]	validation_0-logloss:0.19691
[123]	validation_0-logloss:0.19576
[124]	validation_0-logloss:0.19463
[125]	validation_0-logloss:0.19348
[126]	validation_0-logloss:0.19229
[127]	validation_0-logloss:0.19127
[128]	validation_0-logloss:0.19054
[129]	validation_0-logloss:0.18933
[130]	validation_0-logloss:0.18825
[131]	validation_0-logloss:0.18739
[132]	validation_0-logloss:0.18659
[133]	validation_0-logloss:0.18569
[134]	validation_0-logloss:0.18461
[135]	validation_0-logloss:0.18344
[136]	validation_0-l

...

[122]	validation_0-logloss:0.13147
[123]	validation_0-logloss:0.13147
[124]	validation_0-logloss:0.13147
[125]	validation_0-logloss:0.13147
[126]	validation_0-logloss:0.13147
[127]	validation_0-logloss:0.13147
[128]	validation_0-logloss:0.13147
[129]	validation_0-logloss:0.13147
[130]	validation_0-logloss:0.13147
[131]	validation_0-logloss:0.13147
[132]	validation_0-logloss:0.13147
[133]	validation_0-logloss:0.13147
[134]	validation_0-logloss:0.13147
[135]	validation_0-logloss:0.13147
[136]	validation_0-logloss:0.13147
[137]	validation_0-logloss:0.13147
[138]	validation_0-logloss:0.13147
[139]	validation_0-logloss:0.13147
[140]	validation_0-logloss:0.13147
[141]	validation_0-logloss:0.13147
[142]	validation_0-logloss:0.13147
[143]	validation_0-logloss:0.13147
[144]	validation_0-logloss:0.13147
[145]	validation_0-logloss:0.13147
[146]	validation_0-logloss:0.13147
[147]	validation_0-logloss:0.13163
[148]	validation_0-logloss:0.13163
[149]	validation_0-logloss:0.13163
[150]	validation_0-l

c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()
c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "early_stopping_rounds", "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[I 2026-01-06 22:18:39,124] Trial 43 pruned. Trial was pruned at iteration 30.
[I 2026-01-06 22:18:39,148] Trial 44 pruned. Trial was pruned at iteration 0.
[I 2026-01-06 22:18:39,171] Trial 45 pruned. Trial was pruned at iteration 0.
[0]	validation_0-logloss:0.49673
[1]	validation_0-logloss:0.40298
[2]	validation_0-logloss:0.32519
[3]	validation_0-logloss:0.27532
[4]	validation_0-logloss:0.24001
[5]	validation_0-logloss:0.21517
[6]	validation_0-logloss:0.18966
[7]	validation_0-logloss:0.17648
[8]	validation_0-logloss:0.17089
[9]	validation_0-logloss:0.16442
[10]	validation_0-logloss:0.16297
[11]	validation_0-logloss:0.16166
[12]	validation_0-logloss:0.15738
[13]	validation_0-logloss:0.15406
[14]	validation_0-logloss:0.15533
[15]	validation_0-logloss:0.15444
[16]	validation_0-logloss:0.15526
[17]	validation_0-logloss:0.15262
[18]	validation_0-logloss:0.15546
[19]	validation_0-logloss:0.15828
[20]	validation_0-logloss:0.16201
[21]	validation_0-logloss:0.16216
[22]	validation_0-logloss:0

c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:53: No visible GPU is found, setting device to CPU.
  self.starting_round = model.num_boosted_rounds()
c:\Users\m-rog\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [22:18:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:207: Device is changed from GPU to CPU as we couldn't find any available GPU on the system.
  self.starting_round = model.num_boosted_rounds()



Final Test Accuracy: 0.9561
